## Use financialmodelingprep to obtain fundemental stock info
See the api docs at: https://financialmodelingprep.com/developer/docs/

The financialmodelingprep.com api allows you to capture useful stock fundemental information with 


In [1]:
import pandas as pd
import numpy as np
import requests
import json
from IPython import display



### access financialmodelingprep
For obtaining more info, add to the 2 dictionaries below.  

In [2]:
# fmp_routes is the main url route to obtain various kinds of info
fmp_routes = {
    'profile':'company/profile',
    'ratios':'financial-ratios',
}

# fmp_cols is a dictionary that defines columns to build in the returned DataFrame
#  of the function build_fmp_df (below).
# The main_key key is the top level key of the json data that financialmodelingprep
#  returns from their api calls.
# The col_list key provides columns that you want to add to your resulting DataFrame.
#  Each value in the col_list array has sub keys of the returned json that are separated
#   by '_' characters.  
#   For example:
#    The key ratios_investmentValuationRatios_dividendYield will cause build_fmp_df to find the data at:
#         returned_json['ratiosList']['ratios']['investmentValuationRatios']['dividendYield']
#   The key 'profile_price' will cause build_fmp_df to find the data at: 
#         returned_json['companyProfiles']['profile']['price']
#
fmp_cols = {
    'ratios':{'main_key':'ratiosList',
              'col_list':[
                'symbol',
                'ratios_investmentValuationRatios_dividendYield',
                'ratios_investmentValuationRatios_priceToBookRatio',
                'ratios_debtRatios_debtEquityRatio',
              ]
             },
    'profile':{'main_key':'companyProfiles',
               'col_list':[
                'symbol',
                'profile_price',
                'profile_beta',
                'profile_lastDiv',
               ]
              }
}
    
def build_fmp_df_from_json(fmp_json,fmp_cols_key):
    fmp_cols_dict =  fmp_cols[fmp_cols_key]
    fmp_main_key = fmp_cols_dict['main_key']
    cols = fmp_cols_dict['col_list']
    fmp_data_list = fmp_json[fmp_main_key]
    dict_df_list = []
    for fmp_data in fmp_data_list:
        dict_df = {}
        for c in cols:
            if '_' not in c:
                dict_df[c] = fmp_data[c]
            else:
                subcols = c.split('_')
                sub_dict = fmp_data.copy()
                for subcol in subcols:
                    if type(sub_dict) is list:
                        sub_dict = sub_dict[0]
                    sub_dict = sub_dict[subcol]
                
                dict_df[subcols[-1]] = sub_dict
        dict_df_list.append(dict_df)
    return pd.DataFrame(dict_df_list)



def build_fmp_df_type(fmp_type,symbol_list):
    '''
    Build a data frame of fundemental info (of type fmp_type) 
       for each stock in symbol_list.
    :param fmp_type: one of the keys in fmp_routes or fmp_cols above
    :param symbol_list: a list like ['aapl','msft','amzn','fb','googl']
    
    
    '''
    route = fmp_routes[fmp_type]
    df_final = None
    # the free api only accepts 3 stocks at a time, so call the api on sets of 3
    for i in np.arange(0,len(symbol_list),3):
        clist_string = ','.join([c.upper() for c in symbol_list[i:i+3]])
        fmp_url = f'https://financialmodelingprep.com/api/v3/{route}/{clist_string}'
        frs =  requests.get(fmp_url).json()
        dft = build_fmp_df_from_json(frs,fmp_type)
        if df_final is None:
            df_final = dft.copy()
        else:
            df_final = df_final.append(dft,ignore_index=True)
    return df_final    
    
def build_fmp_df(symbol_list):
    df_all = None
    for t in fmp_routes.keys():
        dft = build_fmp_df_type(t,symbol_list)
        if df_all is None:
            df_all = dft.copy()
        else:
            df_all = df_all.merge(dft,on='symbol',how='inner')
    return df_all

### run example to see all data

In [3]:
clist = ['aapl','msft','amzn','fb','googl']
display.display(build_fmp_df(clist))

,symbol,price,beta,lastDiv,dividendYield,priceToBookRatio,debtEquityRatio
0,AAPL,229.24,1.228499,2.92,0.014628458229615355,10.666314697418441,2.7410043320661304
1,MSFT,137.35,1.231028,1.84,0.014553265436219036,9.27388488224372,1.8003127137691781
2,AMZN,1846.09,1.516863,0,,14.181604898485336,2.6295198195294875
3,FB,149.73,1.062394,0,,5.63999247926851,0.3198487937142518
4,GOOGL,1068.21,1.022765,0,,2.2180341220798043,0.369669681595695


## END